In [103]:
# import libraries
import numpy as np
import cv2 as cv

In [128]:
# read in single test image
img = cv.imread("/home/leo/Documents/UNI/ICDS/Groupwork/Group1/data/test/normal/IM-0115-0001.jpeg", cv.IMREAD_GRAYSCALE)
print(img.shape)

(1858, 2090)


In [151]:
class conv_layer:
    """
    Convolution layer expects an input dimension (in_dim) of shape (h, w, d), where h and 
    are hight and width of images and d the number of dimensions.

    in_dim -> np.array of shape (h,w,d) -> input image dimensions 
    conv_size -> tuple of shape (h, w) -> size of convolution kernel
    kernel_num -> int -> number of kernels
    """
    def __init__(self, in_dim, conv_size=(3,3), kernel_num=4, debug = False):
        self.kernel_num = kernel_num
        self.conv_size = conv_size
        self.conv_kernels = [None] * self.kernel_num

        for i in range(0, self.kernel_num):
            if debug:
                self.conv_kernels[i] = self.debug_conv((conv_size[0], conv_size[1], in_dim[2]))
            else:
                self.conv_kernels[i] = np.empty((conv_size[0], conv_size[1], in_dim[2]))

        self.in_dim = in_dim
        self.out_dim = (self.in_dim[0] - (conv_size[0] - 1), self.in_dim[1] - (self.conv_size[1] - 1), self.kernel_num)
        
    def forward(self, img):
        out_img = np.zeros(self.out_dim)
        for k in range(0, self.kernel_num):
            for h in range(0, self.out_dim[0]):
                for w in range(0, self.out_dim[1]):
                    out_img[h, w, k] = np.sum(img[h:h+self.conv_size[0], w:w+self.conv_size[1],:] * self.conv_kernels[k])
        return out_img

    def get_out_dim(self):
        return self.out_dim
    
    def debug_conv(self, size):
        kernel = np.zeros(size)
        for i in range(0, size[0],2):
            kernel[i,:,:] = 1
        return kernel


In [157]:
# test conv layer
a = np.zeros((6,6,2))
for w in range(0,a.shape[0], 2):
    for h in range(0,a.shape[1]):
        a[w, h, 0] = 1

for w in range(0,a.shape[0]):
    for h in range(0,a.shape[1], 2):
        a[w, h, 1] = 1

print("input before conv")
print(a[:,:,0])
print(a[:,:,1])
c = conv_layer(in_dim=a.shape, conv_size=(3,3), kernel_num=3, debug=True)
print("expected out dim")
print(c.get_out_dim())
d = c.forward(a)
print("output conv shape")
print(d.shape)
print("output conv")
print(d[:,:,1])


input before conv
[[1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0.]]
[[1. 0. 1. 0. 1. 0.]
 [1. 0. 1. 0. 1. 0.]
 [1. 0. 1. 0. 1. 0.]
 [1. 0. 1. 0. 1. 0.]
 [1. 0. 1. 0. 1. 0.]
 [1. 0. 1. 0. 1. 0.]]
expected out dim
(4, 4, 3)
output conv shape
(4, 4, 3)
output conv
[[10.  8. 10.  8.]
 [ 4.  2.  4.  2.]
 [10.  8. 10.  8.]
 [ 4.  2.  4.  2.]]


In [167]:
class max_pooling_layer:
    """
    Max pooling layer

    in_dim -> np.array of shape (h,w,d) -> input image dimensions 
    pooling_size -> tuple of shape (h, w) -> size of pooling filter
    """
    def __init__(self, in_dim, pooling_size=(3,3)):
        self.pooling_size = pooling_size
        self.in_dim = in_dim
        self.out_dim = (self.in_dim[0] - (self.pooling_size[0] - 1), self.in_dim[1] - (self.pooling_size[1] - 1), self.in_dim[2])
        
    def forward(self, img):
        out_img = np.empty(self.out_dim)
        
        for d in range(0, self.out_dim[2]):
            for w in range(0, self.out_dim[0]):
                for h in range(0, self.out_dim[1]):
                    out_img[w,h,d] = np.max(img[w:w+self.pooling_size[0], h:h+self.pooling_size[1],d])
        return out_img
    
    def get_out_dim(self):
        return self.out_dim

In [168]:
# test pool layer
a = np.zeros((5, 5, 3))
index = 0
for w in range(0,a.shape[0]):
    for h in range(0,a.shape[1]):
        a[w,h,:] = index
        index += 1
print("input before pool")
print(a[:,:,0])
c = max_pooling_layer(in_dim=a.shape, pooling_size=(3,3))
d = c.forward(a)
print("output pool")
print(d[:,:,0])

input before pool
[[ 0.  1.  2.  3.  4.]
 [ 5.  6.  7.  8.  9.]
 [10. 11. 12. 13. 14.]
 [15. 16. 17. 18. 19.]
 [20. 21. 22. 23. 24.]]
output pool
[[12. 13. 14.]
 [17. 18. 19.]
 [22. 23. 24.]]


In [109]:
# currently only ReLU activation function hardcoded
class activation_layer:
    def __init__(self, in_dim):
        self.in_dim = in_dim

    def forward(self, img):
        out_img = np.stack(np.vectorize(self.relu)(img), axis=0)
        return out_img

    def relu(self, el):
        return(np.maximum(0, el))

In [110]:
# test activation layer
a = np.zeros((2, 3, 3))
index = -4
for k in range(0,a.shape[1]):
    for h in range(0,a.shape[2]):
        a[:, k, h] = index
        index += 1
print("input before activation")
print(a)
r = activation_layer(a.shape)
b = r.forward(a)
print(b)


input before activation
[[[-4. -3. -2.]
  [-1.  0.  1.]
  [ 2.  3.  4.]]

 [[-4. -3. -2.]
  [-1.  0.  1.]
  [ 2.  3.  4.]]]
[[[0. 0. 0.]
  [0. 0. 1.]
  [2. 3. 4.]]

 [[0. 0. 0.]
  [0. 0. 1.]
  [2. 3. 4.]]]


In [116]:
class fully_connected_layer:
    def __init__(self, in_dim, out_dim):
        self.in_dim = in_dim
        self.out_dim = out_dim
        w_dim = 1
        for d in in_dim:
            w_dim = w_dim * d
        self.weights = np.ones((self.out_dim, w_dim))
    
    def forward(self, img):
        out_vec = np.zeros(self.out_dim)
        img_vec = img.flatten()

        for i in range(0, self.out_dim):
            out_vec[i] = np.sum(img_vec * self.weights[i])
        return out_vec

In [117]:
# test fully connected layer
a = np.ones((2, 3, 3))

print("input before fully connected")
print(a)
c = fully_connected_layer(in_dim=a.shape, out_dim=3)
d = c.forward(a)
print("output pool")
print(d)

input before fully connected
[[[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]]
output pool
[18. 18. 18.]


In [125]:
# test them in succession

img = np.ones((6, 6))
index = 0
for k in range(0,img.shape[0]):
    for h in range(0,img.shape[1]):
        img[k, h] = index
        index += 1

c1 = conv_layer(in_dim=img.shape, conv_size=(3,3), kernel_num=2)
a1 = activation_layer(in_dim=c1.get_out_dim())
p1 = max_pooling_layer(in_dim=c1.get_out_dim(), pooling_size=(3,3))
f1 = fully_connected_layer(in_dim=p1.get_out_dim(), out_dim=2)
print("before cnn")
print(img)
k = c1.forward(img)
print("after conv")
print(k)
k = a1.forward(k)
print("after activation")
print(k)
k = p1.forward(k)
print("after pooling")
print(k)
k = f1.forward(k)
print("after fully connected")
print(k)



before cnn
[[ 0.  1.  2.  3.  4.  5.]
 [ 6.  7.  8.  9. 10. 11.]
 [12. 13. 14. 15. 16. 17.]
 [18. 19. 20. 21. 22. 23.]
 [24. 25. 26. 27. 28. 29.]
 [30. 31. 32. 33. 34. 35.]]
after conv
[[[9.145300e+05 1.007242e+06 1.099954e+06 1.192666e+06]
  [1.470802e+06 1.563514e+06 1.656226e+06 1.748938e+06]
  [2.027074e+06 2.119786e+06 2.212498e+06 2.305210e+06]
  [2.583346e+06 2.676058e+06 2.768770e+06 2.861482e+06]]

 [[1.320000e+03 1.482000e+03 1.644000e+03 1.806000e+03]
  [2.292000e+03 2.454000e+03 2.616000e+03 2.778000e+03]
  [3.264000e+03 3.426000e+03 3.588000e+03 3.750000e+03]
  [4.236000e+03 4.398000e+03 4.560000e+03 4.722000e+03]]]
after activation
[[[9.145300e+05 1.007242e+06 1.099954e+06 1.192666e+06]
  [1.470802e+06 1.563514e+06 1.656226e+06 1.748938e+06]
  [2.027074e+06 2.119786e+06 2.212498e+06 2.305210e+06]
  [2.583346e+06 2.676058e+06 2.768770e+06 2.861482e+06]]

 [[1.320000e+03 1.482000e+03 1.644000e+03 1.806000e+03]
  [2.292000e+03 2.454000e+03 2.616000e+03 2.778000e+03]
  [3.264